# Manual Testing

See `readme.md` for setup instructions.

Then make sure this jupyter notebook is restarted before rebuilding, or `maturin` will fail to copy the fresh `.pyd` library into `.env/Lib/megalinref/` .

In [ ]:
import json
import megalinref as mlr
from pathlib import Path
import pandas as pd
import numpy as np
import util.unit_conversion as uc
import math

In [ ]:
path_to_temp_data = Path(".").resolve().joinpath("../tests/temporary_test_data")
path_to_temp_data.mkdir(exist_ok=True)
path_to_gitignore = path_to_temp_data / ".gitignore"
path_to_json_data = path_to_temp_data / "road_network.json"
path_to_binary_data = path_to_temp_data / "road_network.bin"

## Make `.gitignore`

In [ ]:
path_to_gitignore.write_text("*\n!.gitignore")

## JSON Cache

In [ ]:
if not path_to_json_data.exists():
    print(f"There is no data cached at {path_to_json_data}")
    # Fetch new data
    cached_json_data = mlr.download_fresh_data_as_json()
    # Cache the data
    with path_to_json_data.open("w") as f:
        json.dump(cached_json_data, f)
else:
    # Use cached data
    with path_to_json_data.open("r") as f:
        cached_json_data = json.load(f)

In [ ]:
cached_json_data["features"][1]

In [ ]:
df = pd.DataFrame.from_records([item["properties"] for item in cached_json_data["features"]])

In [ ]:
df.duplicated().sum()

In [ ]:
cways = set()
network_types = set()
for item in cached_json_data["features"]:
    cways.add(item["properties"]["CWY"])
    network_types.add(item["properties"]["NETWORK_TYPE"])
print(cways)
print(network_types)

## Binary Cache

In [ ]:
if not path_to_binary_data.exists():
    print(f"There is no binary data cached at {path_to_binary_data}")
    # Fetch new data
    cached_binary_data = mlr.Lookup.from_dict(cached_json_data).to_binary()
    # Cache the data
    with path_to_binary_data.open("wb") as f:
        f.write(cached_binary_data)
else:
    # Use cached data
    with path_to_binary_data.open("rb") as f:
        cached_binary_data = f.read()

In [ ]:
lookup = mlr.Lookup.from_binary(cached_binary_data)
lookup

In [ ]:
result = lookup.road_slk_from_coordinate(
    lat = -31.89006203575722,
    lon = 115.80183730752809,
    carriageways  = mlr.Cwy["All"],
    network_types = mlr.NetworkType["All"],
)
result